# Packages

In [1]:
import os
import tqdm
import random 
import pandas as pd

from PIL import Image

# Global Variables

In [2]:
PATH_DATA = "data"
PATH_WATERMARKS = os.path.join(PATH_DATA,"watermark")
PATH_IMAGES_CASES = os.path.join(PATH_DATA,"cases")
PATH_CSV_OUTPUTS = os.path.join(PATH_DATA,"csv_outputs")

# Functions

## Function to add watermark to the image

In [3]:
def apply_watermark(image, watermark, transparency=1.0):
    # Cria uma cópia da marca d'água para garantir que a original não seja alterada
    watermark_copy = watermark.copy()

    if image.mode != 'RGBA':
        image = image.convert('RGBA')
    if watermark_copy.mode != 'RGBA':
        watermark_copy = watermark_copy.convert('RGBA')

    # Modifica a transparência apenas na cópia
    alpha = watermark_copy.split()[3]
    alpha = Image.eval(alpha, lambda a: int(a * transparency))
    watermark_copy.putalpha(alpha)

    # Redimensiona a cópia da marca d'água
    scale_factor = random.uniform(0.1, 0.5)  
    new_size = (int(watermark_copy.width * scale_factor), int(watermark_copy.height * scale_factor))
    watermark_copy = watermark_copy.resize(new_size, Image.Resampling.LANCZOS)

    # Calcula a posição e aplica a marca d'água na imagem
    position = ((image.width - watermark_copy.width) // 2, (image.height - watermark_copy.height) // 2)
    combined = Image.alpha_composite(image, Image.new('RGBA', image.size))
    combined.paste(watermark_copy, position, watermark_copy)

    return combined

# Oppening Watermarks

Important Variables: 
- **watermarks_dict**: 
    - Dictionary where the key contains the watermark path and the value contains the loaded watermark

In [4]:
watermarks_dict = {}
for watermark_name in tqdm.tqdm(os.listdir(PATH_WATERMARKS)):
    watermark_path = os.path.join(PATH_WATERMARKS, watermark_name)
    watermark = Image.open(watermark_path)
    
    watermarks_dict[watermark_name] = watermark


100%|██████████| 2/2 [00:00<00:00, 49.05it/s]


# Uploading images and placing watermarks

Important Variables: 
- **images_path_df**: 
    - Dataframe that contains the images paths. 
    - We will use this DataFrame to control the path of the images that will be created.
    - We will also save this dataframe whenever it is changed, so we don't have to run this code again

## Creating Original Dataframe

In [5]:
def get_watermark_image(image_path, watermark,i, transparency=1.0):
    image = Image.open(image_path)
    # Aplicando a marca d'água
    image_watermark = apply_watermark(image, watermark, transparency=transparency)
    # Extrai o nome do arquivo e a extensão
    path, type_ = image_path.rsplit(".", 1)
    watermark_path = f"{path}_watermark_{i}.{type_}"

    # Verifica o tipo de arquivo e ajusta se necessário
    if type_.lower() == 'jpeg' or type_.lower() == 'jpg':
        if image_watermark.mode == 'RGBA':
            # Converte para RGB se for salvar como JPEG
            image_watermark = image_watermark.convert('RGB')
    image_watermark.save(watermark_path)
    return watermark_path

In [7]:
results = []
for case_name in tqdm.tqdm(os.listdir(PATH_IMAGES_CASES)):
    case_path = os.path.join(PATH_IMAGES_CASES, case_name)
    result = [int(case_name.split("_")[-1])]
    for image_name in os.listdir(case_path):
        image_path = os.path.join(case_path, image_name)
        result.append(image_path)
        for i, (watermark_path, watermark) in enumerate(watermarks_dict.items()):
            image_watermark_path = get_watermark_image(image_path, watermark,i, transparency=30)
            result.append(image_watermark_path)
    results.append(result)

100%|██████████| 97/97 [00:12<00:00,  7.80it/s]


In [8]:
result

[96,
 'data\\cases\\case_96\\case_96__image_0.jpg',
 'data\\cases\\case_96\\case_96__image_0_watermark_0.jpg',
 'data\\cases\\case_96\\case_96__image_0_watermark_1.jpg',
 'data\\cases\\case_96\\case_96__image_1.jpg',
 'data\\cases\\case_96\\case_96__image_1_watermark_0.jpg',
 'data\\cases\\case_96\\case_96__image_1_watermark_1.jpg']

In [9]:
images_path_df = pd.DataFrame(results, columns=["CASE", "IMAGE_1_PATH","IMAGE_1_WATERMARK_1_PATH","IMAGE_1_WATERMARK_2_PATH",  "IMAGE_2_PATH", "IMAGE_2_WATERMARK_1_PATH", "IMAGE_2_WATERMARK_2_PATH"]).sort_values("CASE")
images_path_df.to_csv(os.path.join(PATH_CSV_OUTPUTS, "images_path.csv"), index=False)

In [10]:
images_path_df

,CASE,IMAGE_1_PATH,IMAGE_1_WATERMARK_1_PATH,IMAGE_1_WATERMARK_2_PATH,IMAGE_2_PATH,IMAGE_2_WATERMARK_1_PATH,IMAGE_2_WATERMARK_2_PATH
0,0,data\cases\case_0\case_0__image_0.jpg,data\cases\case_0\case_0__image_0_watermark_0.jpg,data\cases\case_0\case_0__image_0_watermark_1.jpg,data\cases\case_0\case_0__image_1.jpg,data\cases\case_0\case_0__image_1_watermark_0.jpg,data\cases\case_0\case_0__image_1_watermark_1.jpg
1,1,data\cases\case_1\case_1__image_0.jpg,data\cases\case_1\case_1__image_0_watermark_0.jpg,data\cases\case_1\case_1__image_0_watermark_1.jpg,data\cases\case_1\case_1__image_1.jpg,data\cases\case_1\case_1__image_1_watermark_0.jpg,data\cases\case_1\case_1__image_1_watermark_1.jpg
12,2,data\cases\case_2\case_2__image_0.jpg,data\cases\case_2\case_2__image_0_watermark_0.jpg,data\cases\case_2\case_2__image_0_watermark_1.jpg,data\cases\case_2\case_2__image_1.jpg,data\cases\case_2\case_2__image_1_watermark_0.jpg,data\cases\case_2\case_2__image_1_watermark_1.jpg
23,3,data\cases\case_3\case_3__image_0.jpg,data\cases\case_3\case_3__image_0_watermark_0.jpg,data\cases\case_3\case_3__image_0_watermark_1.jpg,data\cases\case_3\case_3__image_1.jpg,data\cases\case_3\case_3__image_1_watermark_0.jpg,data\cases\case_3\case_3__image_1_watermark_1.jpg
34,4,data\cases\case_4\case_4__image_0.jpg,data\cases\case_4\case_4__image_0_watermark_0.jpg,data\cases\case_4\case_4__image_0_watermark_1.jpg,data\cases\case_4\case_4__image_1.jpg,data\cases\case_4\case_4__image_1_watermark_0.jpg,data\cases\case_4\case_4__image_1_watermark_1.jpg
...,...,...,...,...,...,...,...
92,92,data\cases\case_92\case_92__image_0.jpg,data\cases\case_92\case_92__image_0_watermark_...,data\cases\case_92\case_92__image_0_watermark_...,data\cases\case_92\case_92__image_1.jpg,data\cases\case_92\case_92__image_1_watermark_...,data\cases\case_92\case_92__image_1_watermark_...
93,93,data\cases\case_93\case_93__image_0.jpg,data\cases\case_93\case_93__image_0_watermark_...,data\cases\case_93\case_93__image_0_watermark_...,data\cases\case_93\case_93__image_1.jpg,data\cases\case_93\case_93__image_1_watermark_...,data\cases\case_93\case_93__image_1_watermark_...
94,94,data\cases\case_94\case_94__image_0.jpg,data\cases\case_94\case_94__image_0_watermark_...,data\cases\case_94\case_94__image_0_watermark_...,data\cases\case_94\case_94__image_1.jpg,data\cases\case_94\case_94__image_1_watermark_...,data\cases\case_94\case_94__image_1_watermark_...
95,95,data\cases\case_95\case_95__image_0.jpg,data\cases\case_95\case_95__image_0_watermark_...,data\cases\case_95\case_95__image_0_watermark_...,data\cases\case_95\case_95__image_1.jpg,data\cases\case_95\case_95__image_1_watermark_...,data\cases\case_95\case_95__image_1_watermark_...
